# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [1]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [2]:
STREAM_SIZE = 10000
web_traffic_stream = stream(STREAM_SIZE)

## Data structures

### Libraries

---

The libraries we use here are numpy and mmh3.

Numpy is used for better matrix manipulation, and mmh3's hash function is used to have an easy way to uniformly distribute the IPs coming from the stream.

In [3]:
import numpy
import mmh3

### Counting how many different IPs are there in the stream :

In order to solve the question on how many uniques IPs are there in the stream, we resort to the HyperLogLog Sketch which is specifically designed to count distinct elements. By using the bit representation of the input, slicing it and updating one of the _m_ counters. It is useful due to its speed and accuracy.

The estimation of the unique IPs, have a standard deviation = 1.04E/m^0.5 which, if we take the answer from a stream_size of 100000, the answer is 85864.85, so the expected value is around the  (74.701,68, 97.026,32)

In [4]:
# Returns the position of the first 1 bit from the left in the word until endIndex
def get_index(binary):
    pos = -1
    try:
        pos = binary.index('1')+1
    except(ValueError):
        pos = endIndex
    return pos

The uniqueIPs function takes as input the stream and then it calculates the values to slice the stream (represented in binary) acording to the given m , then it starts a counter of size length m . For the alpha values we took the suggested in the paper from Flajolet (https://hal.archives-ouvertes.fr/file/index/docid/406166/filename/FlFuGaMe07.pdf ), same with the size adjustments to E . Note that we did not calculate the harmonic mean as stated on the lecture, but we used E = alpha*m^2*(sum(2^M[j])-1). To start assigning values and thus counting, the function splits the line in the stream , taking just the IP, then hashes it into binary, then checks that the binary is of 32 bits, and then slices it into a lower and upper part. Then gets the index of the left-most bit of the lower part of the bit, after it assigns the value of M to update between the max of the last M recorded at that position or the p value. Finally, you get E, with the formula described before and adjustments to this E value are made depending on the selection of m (this selection depends on the application that it will be used and the number of expected distinct elements).

In [5]:
def uniqueIPs(stream, m=64, w=64, seed=42):
    
    # Transforming the data
    upper_m = int(numpy.log2(m))
    upperval=int(numpy.power(2,upper_m)-1)
    M = numpy.zeros(m)
    alpha=0
    if (m<32):
        alpha=0.673
    elif (m<64):
        alpha=0.697    
    elif (m<128):
        alpha=0.709
    else:
        alpha=0.7213/(1+ 1.079/m)
    
    # Iterating the stream
    for line in stream:
        log = line.split()
        hashy = mmh3.hash(log[0], seed=seed, signed=False) 
        bin_hash = bin(hashy)[2:]
        while len(bin_hash)<32:
            bin_hash='0'+bin_hash
        upper = bin_hash[:upper_m]
        lower = bin_hash[upper_m:]
        p = get_index(lower)
        j = int(upper, 2)
        M[j] = max(M[j], p)
        
    # Getting the value
    suma=0
    for j in range(m):
        suma+=numpy.power(2,-M[j])

    E = alpha*numpy.power(m,2)*numpy.power(suma,-1)
    
    if (E<=5*m/2):
        V=numpy.count_nonzero(M==0)
        if (V!=0):
            E_a=m*numpy.log(m/V)
        else:
            E_a=E
    elif (E<=(numpy.power(2,32)/30)):
        E_a=E
    else:
        E_a=-2*numpy.log(1-E/numpy.power(2,32))
        
    return E_a

In [6]:
uniq_ips = uniqueIPs(web_traffic_stream,m=32)
print("There are aprox. "+str(uniq_ips)+" unique IPs in the stream.")

There are aprox. nan unique IPs in the stream.


D:\Programmes\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in double_scalars
D:\Programmes\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in log


### Counting how many different IPs for each domain are there in the stream :

To count the unique IPs seen on each of the domains we have to implement again the HyperLogLog algorithm but instead of having just M counters, we will have M arrays of counters for the distinct elements on each of the domains.

### Counting how many times a certain IP visited a certain domain :

   To know how many times an IP visited a domain, we firstly need to build a data structure that allows us to know how much time each IP visited each domain. 
   This data structure will cause the query to be very efficient, as we will only need to go through a slim part of the data structure to answer the query.
   
   Here, we use the data structure of a CountMin sketch, as it is an efficient way to count the frequency of elements in a stream without storing all the elements.
   The main issue we fall into when implementing this sketch is that here, we have an unknown number of domains, where the CountMin squetch is not initially made to differentiate the elements. 
   What we did to solve this problem is that we created a dictionnary that relates the domain name to its own CountMin Data Structure.
   
   Given that we work with a stream, we had to increment the dictionnary on the go with new domains every time they appear to solve this issue.

In [7]:
def IP_frequency(stream, d, w):
    domains_dict = {}
    for i in stream:
        current_element = i.split()
        current_IP = current_element[0]
        current_domain = current_element[1]
        if current_domain in domains_dict:
            for j in range(d):
                hashw = mmh3.hash(current_IP, j) % w
                domains_dict[current_domain][j, hashw] += 1
        else:
            domains_dict[current_domain]= numpy.zeros((d,w))
            for j in range(d):
                hashw = mmh3.hash(current_IP, j) % w
                domains_dict[current_domain][j, hashw] += 1
    return domains_dict

Now that we have a working Data Structure, we want to have a look at it and see how it interacts with the different parameters : the imput stream size we chose, the number of rows in our matrix and the number of values our hash function can take:

In [8]:
IP_Frequency_Test_Stream = stream(10)
IP_frequency(IP_Frequency_Test_Stream, 4, 16)

{'python.org': array([[0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 2., 1.]]),
 'wikipedia.org': array([[0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 2., 0., 0., 0., 0.],
        [0., 2., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.]]),
 'pandas.pydata.org': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])}

**Defining the query**

Now that we have our database, we can go and use it to answer our initial question : How many time was a certain IP seen on a certain domain ?

To do that, we go through each line of the matrix associated to our domain and build a list out of the values from our ip. The functiun then returns the minimum of this value, as originaly intended in the CountMin squetch.

In [9]:
def CountMin(stream, ip, domain, d, w):
    Count_List = []
    Domains_Dict = IP_frequency(stream, d, w)
    if domain in Domains_Dict:
        M = Domains_Dict[domain]
        for k in range(d):
            Count_List.append(M[k, mmh3.hash(ip, k) % w])
    else:
        Count_List.append(0)
    return min(Count_List)

In order to check the accuracy of our query, we can use a function that will have 100% accuracy while being absolutely inefficient :

In [10]:
def Accurate_IP_Count(stream, ip, domain):
    count = 0
    for i in stream:
        current_element = i.split()
        current_IP = current_element[0]
        current_domain = current_element[1]
        if (current_IP == ip) & (current_domain == domain):
            count += 1
    return count

To have an idea about the result and the accuracy we can expect from out squetch, we firstly made a blind try with arbitrary values :

In [11]:
Test_IP = '202.152.82.171'
Test_Domain = 'wikipedia.org'
CountMin_Test_Stream = stream(STREAM_SIZE)

print('The estimated number of times Test_IP went on Test_Domain is :')
print(CountMin(CountMin_Test_Stream, Test_IP, Test_Domain, 10,1000))

CountMin_Check_Stream = stream(STREAM_SIZE)

print('The exact number of times Test_IP went on Test_Domain is :')
print(Accurate_IP_Count(CountMin_Check_Stream, Test_IP, Test_Domain))

The estimated number of times Test_IP went on Test_Domain is :
3.0
The exact number of times Test_IP went on Test_Domain is :
1


Obviously, this first answer does not seem to be even close from the result we expected. We now need to look for the best values we can give $d$ and $w$ in order to have the highest chance of getting the most precise value.

According to *An Improved Data Stream Summary:The Count-Min Sketch and its Applications*, the number of values our hash function can take, $w$, is tied to $\epsilon$, the factor defining the error, and $d$ is linked to $\delta$, the probability of the error being in that $\epsilon$ factor.

Finding a correct value for $d$ is a rather easy task. We have : 

$d=\ln(\frac{1}{\delta})$ so, if we want the error rate to be less than 0.01%, we find that $d$ should be at least 9.22, thus rounding up $d$ to $d=10$ fits the probability we asked for.

To find the value $\epsilon$ that fits our needs, we need to have a deeper reflexion than simply asking us what the probability $\delta$ we want.
Indeed, with each iteration of the stream, the error cumulates, and so we need to account for that when we choose our $w$ dimension. We have :

* $f_e \leq f'_e \leq f_e \cdot \epsilon \cdot N$, with $f_e$ the real frequency, $f'_e$ the estimated frequency and $N$ the number of items we have read in the stream.

* $w = \frac{e}{\epsilon}$

Thus, $f_e \leq f'_e \leq f_e \cdot \frac{e}{w} \cdot N$ 

And thus we can chose w depending on the number of items in the stream, and the error interval we want. Tipically, if you want your approximation to be at most 10% off of the real value, you want to chose $w$ such that $\frac{e\cdot N}{w} = 1.1$, which gives us $w \approx 2.5 \cdot N$

Of course, building a Data Structure that is bigger than the stream makes no sense as using streaming algorithms would be pointless, so we want to take $w$ as a fraction of $N$, depending on our precesion needed, our processing time available and our memory available.

For that, it seemed that $w = \frac{N}{10}$ could be a good compromise.

### Counting how many times a certain IP visited a list of domains:

To implement this query, we use the same Data Structure as for a single domain, but the query is a bit more complex :

We use the fact that indepently of the domain, hashing an IP will always give the same result, and so adding up the values from each domain line per line, or hash seed by hash seed, allows us to minimize the error by not counting it every time.

In [12]:
def Count_Multiple_Min(stream, ip, domains, d, w):
    Count_List = []
    Domains_Dict = IP_frequency(stream, d, w)
    for k in range(d):
        Sum = 0
        for d in domains:
            if d in Domains_Dict:
                Sum += Domains_Dict[d][k, mmh3.hash(ip, k) % w]
# For clarity purpose, one could add this condition:
            #else:
            #    Sum += 0
        Count_List.append(Sum)
    return min(Count_List)

Here, doing a step-by-step analysis to calculate the parameters would be a rather complicated task, but we could argue that having limited the times we add the error, the final error could be proportionnal to the individual error.
Thus, taking the same parameters as in the single-domain query would lead to an error that, even if bigger, still corresponds to the magnitude of the single-domain error. 

In [13]:
Multiple_Domains_Stream = stream(STREAM_SIZE)
Test_Domains = ['wikipedia.org','python.org','lol.lol','github.com']

Count_Multiple_Min(Multiple_Domains_Stream, Test_IP, Test_Domains, 10, 1000)

5.0

## Analysis

Having tried to find how many times IP X went on a few differents domains, we have found that when accounting error rates, every single IP we have tried appeared less than just a few times, meaning that those queries are, given this stream set, not that usefull since it does not give any valuable information.